### Imports

In [18]:
# LLM
from langchain_community.llms import Ollama

# Document loader
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Embedding
from langchain_community.embeddings.ollama import OllamaEmbeddings

# Vector store
from langchain.vectorstores import SKLearnVectorStore

# Prompt
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

MODEL = 'llama3.1'
DATA_PATH = 'data'

In [19]:
model = Ollama(model = MODEL)
embeddings = OllamaEmbeddings(model = 'nomic-embed-text')

### Reading and chunking the document

In [20]:
loader = PyPDFDirectoryLoader(DATA_PATH)
documents = loader.load()
        
text_spitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 250,
    chunk_overlap = 0
)

chunks = text_spitter.split_documents(documents)

### Creating the Vector Store

In [21]:
vector_store = SKLearnVectorStore.from_documents(
    documents = chunks,    
    embedding = embeddings
)

### Using LLama with RAG

In [22]:
retriever = vector_store.as_retriever(k = 4)

prompt_template = """
    Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    Answer in Portuguese.

    Use three sentences maximum and keep the answer concise. 
    Question: {question}
    =========
    Documents: {documents}
    =========
    Answer:
"""

PROMPT = PromptTemplate(
    template = prompt_template, input_variables = ["question", "documents"]
)

chain = RetrievalQA.from_chain_type(
    llm = model,
    retriever = retriever,
    chain_type_kwargs = {
        'prompt': PROMPT,
        'document_variable_name': 'documents'
    }
)

In [23]:
QUERY = 'Eu posso receber propina na TARRAF?'
response = chain.invoke(QUERY)
resposta = response['result']

In [24]:
print(f'Pergunta: {QUERY}')
print('\n')
print(f'Resposta: {resposta}')

Pergunta: Eu posso receber propina na TARRAF?


Resposta: Não, não é possível receber propina na TARRAF, pois o relacionamento com fornecedores e parceiros é pautado pelo sentido de parceria e cooperação, sendo necessário agir buscando os melhores interesses da empresa. Além disso, a TARRAF preza por uma relação transparente, saudável e de parceria, isenta de favorecimentos.

(SOURCES: tarraf.com.br)
